In [1]:
import syft as sy
from syft.abstract_node import NodeType


kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/eelco/dev/PySyft


In [2]:
low_side = sy.Orchestra.launch(
    name="canada-1",
    node_side_type="low",
    local_db=True,
    reset=True,
#     enable_warnings=True,
)

Consumer service Name:  None
Creating Default Worker Image
Building Default Worker Image
Creating default Worker Pool
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
high_side = sy.Orchestra.launch(
    name="canada-2",
    local_db=True,
    reset=True,
#     enable_warnings=True,
)

Consumer service Name:  None
Creating Default Worker Image
Building Default Worker Image
Creating default Worker Pool
Created default worker pool.
Data Migrated to latest version !!!


In [4]:
client_low_do = low_side.login(email="info@openmined.org", password="changethis")
client_high_do = high_side.login(email="info@openmined.org", password="changethis")

client_low_do.register(name="Sheldon", email="sheldon@caltech.edu", password="changethis", password_verify="changethis")
client_low_ds = low_side.login(email="sheldon@caltech.edu", password="changethis")

Logged into <canada-1: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <canada-2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <canada-1: Low side Domain> as <sheldon@caltech.edu>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [5]:
import numpy as np

mock = np.random.random(10)
real = np.random.random(10)

dataset = sy.Dataset(
        name="my-dataset",
        description="abc",
        asset_list=[
            sy.Asset(
                name="numpy-data",
                mock=mock,
                data=real,
                shape=real.shape,
                mock_is_real=True,
            )
        ],
    )

client_low_do.upload_dataset(dataset)

  0%|                                                     | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  8.99it/s]


SyftSuccess: Dataset uploaded to 'canada-1'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [6]:
import numpy as np

mock = np.random.random(10)
real = np.random.random(10)

dataset = sy.Dataset(
        name="my-dataset",
        description="abc",
        asset_list=[
            sy.Asset(
                name="numpy-data",
                mock=mock,
                data=real,
                shape=real.shape,
                mock_is_real=True,
            )
        ],
    )

client_high_do.upload_dataset(dataset)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  8.49it/s]

Uploading: numpy-data


SyftSuccess: Dataset uploaded to 'canada-2'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
data = client_low_do.datasets[0].assets[0]

@sy.syft_function_single_use(data=data)
def compute_mean(data: np.ndarray) -> float:
    return data.mean()

client_low_ds.code.request_code_execution(compute_mean)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

syft.service.request.request.Request

In [8]:
low_request = client_low_do.requests[0]

In [9]:
from syft.service.code.user_code import UserCode
from syft.service.context import AuthedServiceContext

In [10]:
client_high_do.code.submit(low_request.code)

SyftSuccess: User Code Submitted

In [15]:
client_high_do._fetch_api(client_high_do.credentials)

In [28]:
data_high = client_high_do.datasets[0].assets[0]

client_high_do.code.compute_mean(data=data_high)

SyftError: `data` must be of type `ndarray` not `Asset`

In [27]:
data_high.pointer

```python
TwinPointer(Real)
```
array([0.79384466, 0.08654086, 0.21177165, 0.83140243, 0.62535285,
       0.59940464, 0.65184947, 0.109039  , 0.58450115, 0.1062202 ])